# HomeWork 7
На цьому тижні ми вивчили як працюють рекомендаційні системи. Пропонуємо вам познайомитись з бібліотекою surprise, котра якраз є по суті додатком до знайомої нам бібліотеки scikit-learn для тренування моделей рекомендаційних систем.

Візьміть датасет movielens і побудуйте модель матричної факторизації. У даній бібліотеці він має назву SVD. Підберіть найкращі параметри за допомогою крос-валідації, також поекспериментуйте з іншими алгоритмами розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.

Підказки як саме побудувати дану модель ви знайдете в документації до даної бібліотеки.

In [10]:
from pathlib import Path

import pandas as pd

__OBTAIN & SCRUB__

- links.csv - movieId, imdbId, tmdbId
- movies.csv - movieId, title, genres
- ratings.csv - userId, movieId, rating, timestamp
- tags.csv - userId, movieId, tag, timestamp

In [11]:
def read_from_csvfile(file: Path) -> pd.DataFrame:
    """Read content from csv-file and return dataframe from content."""
    df = pd.read_csv(file)

    return df

In [13]:
# import and Create new folder
import os
from google.colab import files
import shutil

new_folder = 'ml'

if os.path.isdir(new_folder):
  shutil.rmtree(new_folder)

os.mkdir(new_folder)

# Upload Files to GoogleColab
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(new_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

Saving ratings.csv to ratings.csv
move ratings.csv to ml/ratings.csv


In [14]:
links = read_from_csvfile('/content/ml/links.csv')
movies = read_from_csvfile('/content/ml/movies.csv')
ratings = read_from_csvfile('/content/ml/ratings.csv')
tags = read_from_csvfile('/content/ml/tags.csv')

In [ ]:
links.tail(3)

,movieId,imdbId,tmdbId
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0
9741,193609,101726,37891.0


In [ ]:
movies.tail(3)

,movieId,title,genres
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [15]:
ratings.tail(3)

,userId,movieId,rating,timestamp
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [ ]:
tags.tail(3)

,userId,movieId,tag,timestamp
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978
3682,610,168248,Heroic Bloodshed,1493844270


- Використовуємо лише датасет рейтингу, для найпростішої системи

__MODEL__

- модель матричної факторизації. У даній бібліотеці він має назву SVD

- https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD
- https://www.piwheels.org/project/scikit-surprise/
- https://pypi.org/project/scikit-surprise/
- https://github.com/NicolasHug/Surprise

!pip install matplotlib

In [16]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811635 sha256=f7f9a3b54c8d6954ecb6a4aa9e30efbf64ecb663a730ee50b43da5aa3cf0af67
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [17]:
from surprise.prediction_algorithms.matrix_factorization import SVD  # poetry add surprise
# poetry add scikit-surprise==1.1.3 --use-pep517
# !!!! Installing scikit-surprise (1.1.3): Failed
# colab - Ok

- https://surprise.readthedocs.io/en/stable/getting_started.html#cross-validate-example

- https://surprise.readthedocs.io/en/stable/model_selection.html

In [ ]:
# cross_validate(
#                algo,
#                data,
#                measures=['rmse', 'mae'],
#                cv=None,
#                return_train_measures=False,
#                n_jobs=1,
#                pre_dispatch='2*n_jobs',
#                verbose=False
#                )

- https://surprise.readthedocs.io/en/stable/getting_started.html?highlight=Dataset#use-a-custom-dataset

In [18]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

Column names are irrelevant, but specify only one rating per line, and each line needs to respect the following structure:<br> `user ; item ; rating ; [timestamp]`

In [19]:
df = ratings[['userId', 'movieId', 'rating']]

In [20]:
df.tail(3)

,userId,movieId,rating
100833,610,168250,5.0
100834,610,168252,5.0
100835,610,170875,3.0


In [21]:
df['rating'].max(), df['rating'].min(), df['rating'].mean(), df['rating'].std()

(5.0, 0.5, 3.501556983616962, 1.042529239060635)

0.5 мінімальний рейтинг, тому логічно що межа мінімума 0:

- https://surprise.readthedocs.io/en/stable/reader.html?highlight=Reader#surprise.reader.Reader

In [22]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 5))

In [23]:
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [ ]:
algo2 = SVD(
            n_factors=1,
            n_epochs=10,
            biased=True,
            init_mean=2.5,
            init_std_dev=0.5,
            lr_all=0.005,
            reg_all=0.02,
            lr_bu=None,
            lr_bi=None,
            lr_pu=None,
            lr_qi=None,
            reg_bu=None,
            reg_bi=None,
            reg_pu=None,
            reg_qi=None,
            random_state=None,
            verbose=False
            )

In [ ]:
# Run 6-fold cross-validation and print results.
cross_validate(algo2, data, measures=['RMSE', 'MAE'], cv=6, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 6 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Mean    Std     
RMSE (testset)    0.8773  0.8767  0.8783  0.8754  0.8740  0.8764  0.8764  0.0014  
MAE (testset)     0.6781  0.6737  0.6791  0.6732  0.6726  0.6762  0.6755  0.0025  
Fit time          0.65    0.52    0.55    0.50    0.54    0.55    0.55    0.05    
Test time         0.23    0.14    0.12    0.18    0.14    0.33    0.19    0.07    


{'test_rmse': array([0.87733225, 0.87673298, 0.8783383 , 0.87544726, 0.87404887,
        0.87642241]),
 'test_mae': array([0.67806792, 0.67367781, 0.67905478, 0.67322409, 0.67264009,
        0.67622364]),
 'fit_time': (0.6548593044281006,
  0.5241212844848633,
  0.5488119125366211,
  0.5006754398345947,
  0.5435709953308105,
  0.5492165088653564),
 'test_time': (0.2271735668182373,
  0.13544225692749023,
  0.12300491333007812,
  0.18457531929016113,
  0.13525629043579102,
  0.33479881286621094)}

- Підберіть найкращі параметри за допомогою крос-валідації


- поекспериментуйте з іншими алгоритмами розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним

In [ ]:
algo_SVD = {
            f'SVD(nfc{nfs}, neph{eph})':
            SVD(
                n_factors=nfs,
                n_epochs=eph,
                init_mean=df['rating'].mean(), # (0+5)/2 ...
                init_std_dev=df['rating'].std()
                )
            for nfs in (1, 5, 10, 20, 40)
            for eph in (5, 10, 20)
           }

In [ ]:
assessment_SVD = {
                  f'{name}_cv{cv}':
                  cross_validate(alg, data, measures=['RMSE', 'MAE'], cv=cv, verbose=True)
                  for name, alg in algo_SVD.items() for cv in (3, 5, 6)
                  }

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9528  0.9544  0.9487  0.9519  0.0024  
MAE (testset)     0.7410  0.7412  0.7372  0.7398  0.0018  
Fit time          0.14    0.17    0.16    0.16    0.01    
Test time         0.34    1.03    0.22    0.53    0.36    
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9376  0.9333  0.9342  0.9322  0.9348  0.9344  0.0018  
MAE (testset)     0.7273  0.7224  0.7220  0.7215  0.7268  0.7240  0.0025  
Fit time          0.20    0.19    0.23    0.29    0.30    0.24    0.04    
Test time         0.16    0.59    0.25    0.23    0.25    0.30    0.15    
Evaluating RMSE, MAE of algorithm SVD on 6 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Mean    Std     
RMSE (testset)    0.9299  0.9336  0.9305  0.9345  0.9408  0.9389  0.9347  0.0040  
MA

In [ ]:
assessment_SVD

{'SVD(nfc1, neph5)_cv3': {'test_rmse': array([0.95276599, 0.95440022, 0.94866779]),
  'test_mae': array([0.74100039, 0.74116716, 0.73719141]),
  'fit_time': (0.13503575325012207, 0.1697537899017334, 0.16119813919067383),
  'test_time': (0.3430521488189697, 1.029890775680542, 0.21920371055603027)},
 'SVD(nfc1, neph5)_cv5': {'test_rmse': array([0.93757244, 0.93329877, 0.93418413, 0.93216991, 0.93475519]),
  'test_mae': array([0.72730722, 0.72243911, 0.72197543, 0.72146057, 0.72684302]),
  'fit_time': (0.19652342796325684,
   0.1932849884033203,
   0.23344850540161133,
   0.2891273498535156,
   0.29700589179992676),
  'test_time': (0.15636348724365234,
   0.5948758125305176,
   0.24585938453674316,
   0.2341902256011963,
   0.25353264808654785)},
 'SVD(nfc1, neph5)_cv6': {'test_rmse': array([0.92988242, 0.93363267, 0.93051699, 0.93453742, 0.94081693,
         0.9388671 ]),
  'test_mae': array([0.71572571, 0.72319195, 0.72376976, 0.72235872, 0.72347574,
         0.72934517]),
  'fit_time':

In [ ]:
# best_SVD_by_rmse = min(assessment_SVD, key=lambda x: x['test_rmse'].mean())
all_SVD_by_rmse = {
                   name: value['test_rmse'].mean()
                   for name, value in assessment_SVD.items()
                   }
best_SVD_by_rmse = min(all_SVD_by_rmse, key=all_SVD_by_rmse.get)

In [ ]:
best_SVD_by_rmse

'SVD(nfc1, neph20)_cv5'

In [ ]:
# best_SVD_by_mae = min(assessment_SVD, key=lambda x:x['test_mae'].mean())
all_SVD_by_mae = {
                  name: value['test_mae'].mean()
                  for name, value in assessment_SVD.items()
                  }
best_SVD_by_mae = min(all_SVD_by_mae, key=all_SVD_by_mae.get)

In [ ]:
best_SVD_by_mae

'SVD(nfc1, neph20)_cv5'

Для моделі матричної факторизації (SVD) серед деякого набору параметрів виявився найкращим варіант з 1 фактором, 20 епохами при 5 сегментах кросвалідації.

In [26]:
from surprise.model_selection import GridSearchCV

врахуємо й інші деякі гіперпараметри для SVD

In [ ]:
param_grid = {
              'n_factors': (1, 5, 10, 20, 40),
              'n_epochs': (5, 10, 20),
              'init_mean': (df['rating'].mean(), ((df['rating'].max() + df['rating'].min()) / 2)),
              'init_std_dev': (df['rating'].std(), 1),
              'lr_all': (0.002, 0.005),
              'reg_all': (0.4, 0.6)
              }
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=6)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.8822805803745087
{'n_factors': 1, 'n_epochs': 20, 'init_mean': 2.75, 'init_std_dev': 1, 'lr_all': 0.005, 'reg_all': 0.4}


0.8822805803745087<br>
{'n_factors': 1, 'n_epochs': 20, 'init_mean': 2.75, 'init_std_dev': 1, 'lr_all': 0.005, 'reg_all': 0.4}<br>
(17+ хв.оброблення ~240 models)

In [ ]:
# best MAE score
print(gs.best_score['mae'])

# combination of parameters that gave the best MAE score
print(gs.best_params['mae'])

0.6816170614867199
{'n_factors': 1, 'n_epochs': 20, 'init_mean': 2.75, 'init_std_dev': 1, 'lr_all': 0.005, 'reg_all': 0.4}


In [65]:
param_grid = {
              'n_factors': (1, ),
              'n_epochs': (20, ),
              'init_mean': (2.5,),
              'init_std_dev': (0.5,),
              'lr_all': (0.005,),
              'reg_all': (0.4, )
              }
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=6)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

# best MAE score
print(gs.best_score['mae'])

# combination of parameters that gave the best MAE score
print(gs.best_params['mae'])

0.8790633332191824
{'n_factors': 1, 'n_epochs': 20, 'init_mean': 2.5, 'init_std_dev': 0.5, 'lr_all': 0.005, 'reg_all': 0.4}
0.678878893083766
{'n_factors': 1, 'n_epochs': 20, 'init_mean': 2.5, 'init_std_dev': 0.5, 'lr_all': 0.005, 'reg_all': 0.4}


In [66]:
parameters = gs.best_params['rmse']
compare_results_RMSE = {f'''SVD_nf{parameters.get('n_factors')}_nep{parameters.get('n_epochs')}''': gs.best_score['rmse']}
parameters = gs.best_params['mae']
compare_results_MAE = {f'''SVD_nf{parameters.get('n_factors')}_nep{parameters.get('n_epochs')}''': gs.best_score['mae']}
compare_results_fit_time = {f'''SVD_nf{parameters.get('n_factors')}_nep{parameters.get('n_epochs')}''': 7}

In [67]:
compare_results_RMSE, compare_results_MAE

({'SVD_nf1_nep20': 0.8790633332191824}, {'SVD_nf1_nep20': 0.678878893083766})

Для моделі матричної факторизації (SVD) серед деякого набору параметрів виявився найкращим варіант з 1 фактором, 20 епохами за 6 сегментів кросвалідації.

In [68]:
from surprise import SVDpp, NMF

In [27]:
param_grid = {
              'n_factors': (1, 5),
              'n_epochs': (10, 20),
              'init_mean': (df['rating'].mean(), ((df['rating'].max() + df['rating'].min()) / 2)),
              'init_std_dev': (df['rating'].std(),),
              'lr_all': (0.005,),
              'reg_all': (0.4,)
              }
gspp = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=6)

gspp.fit(data)

# best RMSE score
print(gspp.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gspp.best_params['rmse'])

1.8254104497998247
{'n_factors': 1, 'n_epochs': 10, 'init_mean': 3.501556983616962, 'init_std_dev': 1.042529239060635, 'lr_all': 0.005, 'reg_all': 0.4}


In [50]:
# best MAE score
print(gspp.best_score['mae'])

# combination of parameters that gave the best MAE score
print(gspp.best_params['mae'])

1.4984430163830378
{'n_factors': 1, 'n_epochs': 10, 'init_mean': 3.501556983616962, 'init_std_dev': 1.042529239060635, 'lr_all': 0.005, 'reg_all': 0.4}


Для моделі SVD++ серед деякого набору параметрів виявився найкращим варіант з 1 фактором, 10 епохами за 6 сегментів кросвалідації.

46+ хв.оброблення для 32 моделей для одного сегменту - недочекався. 28+ хв. - 8 моделей

In [69]:
param_grid = {
              'n_factors': (1,),
              'n_epochs': (10, ),
              'init_mean': (df['rating'].mean(), 2.5,),
              'init_std_dev': (df['rating'].std(),),
              'lr_all': (0.005,),
              'reg_all': (0.4,)
              }
gspp = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=6)

gspp.fit(data)

# best RMSE score
print(gspp.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gspp.best_params['rmse'])

# best MAE score
print(gspp.best_score['mae'])

# combination of parameters that gave the best MAE score
print(gspp.best_params['mae'])

1.8254252212988684
{'n_factors': 1, 'n_epochs': 10, 'init_mean': 3.501556983616962, 'init_std_dev': 1.042529239060635, 'lr_all': 0.005, 'reg_all': 0.4}
1.498443016383038
{'n_factors': 1, 'n_epochs': 10, 'init_mean': 3.501556983616962, 'init_std_dev': 1.042529239060635, 'lr_all': 0.005, 'reg_all': 0.4}


In [70]:
parameters = gspp.best_params['rmse']
compare_results_RMSE.update({f'''SVDpp_nf{parameters.get('n_factors')}_nep{parameters.get('n_epochs')}''': gspp.best_score['rmse']})
parameters = gspp.best_params['mae']
compare_results_MAE.update({f'''SVDpp_nf{parameters.get('n_factors')}_nep{parameters.get('n_epochs')}''': gspp.best_score['mae']})
compare_results_fit_time.update({f'''SVDpp_nf{parameters.get('n_factors')}_nep{parameters.get('n_epochs')}''': 278 / 2})

In [71]:
compare_results_RMSE, compare_results_MAE

({'SVD_nf1_nep20': 0.8790633332191824, 'SVDpp_nf1_nep10': 1.8254252212988684},
 {'SVD_nf1_nep20': 0.678878893083766, 'SVDpp_nf1_nep10': 1.498443016383038})

In [76]:
param_grid = {
              'n_factors': (1, 5),
              'n_epochs': (10, 20),
              'reg_pu': (0.01, 0.06, 0.1),
              'reg_qi': (0.01, 0.06, 0.1),
              'reg_bu': (0.01, 0.02, 0.03),
              'reg_bi': (0.01, 0.02, 0.03),
              'lr_bu': (0.005,),
              'lr_bi': (0.005,)
              }
gsNMF= GridSearchCV(NMF, param_grid, measures=['rmse', 'mae'], cv=6)

gsNMF.fit(data)

# best RMSE score
print(gsNMF.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gsNMF.best_params['rmse'])

1.1694317006535082
{'n_factors': 5, 'n_epochs': 20, 'reg_pu': 0.1, 'reg_qi': 0.1, 'reg_bu': 0.03, 'reg_bi': 0.03, 'lr_bu': 0.005, 'lr_bi': 0.005}


In [77]:
# best MAE score
print(gsNMF.best_score['mae'])

# combination of parameters that gave the best MAE score
print(gsNMF.best_params['mae'])

0.9854530355525711
{'n_factors': 5, 'n_epochs': 20, 'reg_pu': 0.1, 'reg_qi': 0.1, 'reg_bu': 0.03, 'reg_bi': 0.03, 'lr_bu': 0.005, 'lr_bi': 0.005}


Для моделі NMF серед деякого набору параметрів виявився найкращим варіант з 5 факторами, 20 епохами за 6 сегментів кросвалідації.

(27+ хв.оброблення - 324 моделей)

In [73]:
param_grid = {
              'n_factors': (5,),
              'n_epochs': (20,),
              'reg_pu': (0.1,),
              'reg_qi': (0.1,),
              'reg_bu': (0.03,),
              'reg_bi': (0.03,),
              'lr_bu': (0.002,),
              'lr_bi': (0.002,)
              }
gsNMF= GridSearchCV(NMF, param_grid, measures=['rmse', 'mae'], cv=6)

gsNMF.fit(data)

# best RMSE score
print(gsNMF.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gsNMF.best_params['rmse'])

# best MAE score
print(gsNMF.best_score['mae'])

# combination of parameters that gave the best MAE score
print(gsNMF.best_params['mae'])

1.1731882748295936
{'n_factors': 5, 'n_epochs': 20, 'reg_pu': 0.1, 'reg_qi': 0.1, 'reg_bu': 0.03, 'reg_bi': 0.03, 'lr_bu': 0.002, 'lr_bi': 0.002}
0.9888994568658788
{'n_factors': 5, 'n_epochs': 20, 'reg_pu': 0.1, 'reg_qi': 0.1, 'reg_bu': 0.03, 'reg_bi': 0.03, 'lr_bu': 0.002, 'lr_bi': 0.002}


In [74]:
parameters = gsNMF.best_params['rmse']
compare_results_RMSE.update({f'''NMF_nf{parameters.get('n_factors')}_nep{parameters.get('n_epochs')}''': gsNMF.best_score['rmse']})
parameters = gsNMF.best_params['mae']
compare_results_MAE.update({f'''NMF_nf{parameters.get('n_factors')}_nep{parameters.get('n_epochs')}''': gsNMF.best_score['mae']})
compare_results_fit_time.update({f'''NMF_nf{parameters.get('n_factors')}_nep{parameters.get('n_epochs')}''': 9})

In [75]:
compare_results_RMSE, compare_results_MAE, compare_results_fit_time

({'SVD_nf1_nep20': 0.8790633332191824,
  'SVDpp_nf1_nep10': 1.8254252212988684,
  'NMF_nf5_nep20': 1.1731882748295936},
 {'SVD_nf1_nep20': 0.678878893083766,
  'SVDpp_nf1_nep10': 1.498443016383038,
  'NMF_nf5_nep20': 0.9888994568658788},
 {'SVD_nf1_nep20': 7, 'SVDpp_nf1_nep10': 139.0, 'NMF_nf5_nep20': 9})

При ручній зміні меж сітки параметрів за результатами видно, що при підборі певних параметрів виявлена перевага алгоритму SVD як по точності, так і по швидкості тренування моделі. SVD++ виявився шось занадто повільним та за 20 епохами гірший за 10.